In [1]:
from copy import deepcopy
from collections import defaultdict
import random
import numpy as np
import time 
import json
from sklearn.linear_model import LinearRegression

## Reading and initializing functions

In [2]:
def get_sudokus(filename, sudokusize=9):
    '''
    get_states is to read multiple sudokus in the format:
    ..8..4..5...64..5 etc.
    returns a list of sudoku board states as clauses in a dict
    '''
    sudokus = []
    with open(filename) as sudoku_states:
        for i, state in enumerate(sudoku_states):
            sudoku = {}
            pointers = {}
            row = 1
            index = 0
            for j, content in enumerate(state):
                if j%sudokusize == 0 and j > 0:
                    row+=1
                try:
                    int(content)
                    literal = int(str(row) + str(j%sudokusize+1) + content)
                    sudoku[index] = [literal]
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    index += 1
                except:
                    # is not convertible to int
                    continue
            sudokus.append((sudoku, pointers))
    return sudokus

def get_constraints_from_dimacs(filename, prevConstraints={}, prevPointers={}):
    '''
    Convert dimacs file to a dictionary of constraints
    and pointers. Allows merging with previous pointers and constraints.
    '''
    constraints = prevConstraints # dictionary with constraints as: clausenumber->[literals]
    pointers = prevPointers # dictionary with pointers to clause numbers for literals as: literals->[clausenumbers]
    initial_length = len(constraints)
    with open(filename) as f:
        for i, clause in enumerate(f):
            index = i + initial_length
            if clause[0] == 'c' or clause[0] == 'p':
                # skip comments etc.
                continue
            constraints[index]= []
            for literal in clause.split():
                literal = int(literal)
                if literal != 0:
                    constraints[index].append(literal)
                    if literal not in pointers.keys():
                        pointers[literal] = [index]
                    else:
                        pointers[literal].append(index)
                    
    return constraints, pointers

    

## DP helper functions for constraints updates

In [105]:
def assign(literals, constraints, pointers, assignments, choice=False):
    '''
    assign the given literal to be True.
    assign_literal is aimed to be one recursion stack, so deepcopies are made
    in order to properly backtrack
    '''
    constraintsCopied = deepcopy(constraints)
    pointersCopied = deepcopy(pointers)
    assignmentsCopied = deepcopy(assignments)
    assignmentsUpdated = update_assigments(assignmentsCopied, literals)
    constraintsUpdated, pointersUpdated = update_constraints(constraintsCopied, pointersCopied, literals)
    if choice:
        extract_features(next(iter(literals)), constraints, pointers, assignments)
    return constraintsUpdated, pointersUpdated, assignmentsUpdated
     
def update_constraints(constraints, pointers, literals):
    '''
    literal: e.g. 112 or 112'
    Updates constraints and pointers by removing all clauses 
    with the given literal, because the clause becomes true.
    Remove counter-literal out of all clauses.
    '''
    ## Find clauses with literal and remove them
    for literal in literals:
        clauses=list(pointers[literal])
        for clause in list(pointers[literal]):
            for neighbourLiteral in constraints[clause]:
                pointers[neighbourLiteral].remove(clause)
            del constraints[clause]
        del pointers[literal]
    
    ## Find clauses with counter-literal and remove its counter-literal
    for literal in literals:
        counterLiteral = -literal
        if counterLiteral in pointers.keys():
            counters=list(pointers[counterLiteral])
            for clause in counters:
                constraints[clause].remove(counterLiteral)
            del pointers[counterLiteral]
   
    return constraints, pointers

def update_assigments(assignments, literals):
    '''
    Keep track of assigned literals.
    Needed in order to return satisfiable solution of literals.
    '''
    for literal in literals:
        if literal in assignments:
            print("already assigned", lit, "(in this stack), returning...")
#             return
        else:
            assignments.append(literal)
    return assignments

def check_tautologies(constraints, pointers):
    '''
    Check for clauses with a tautology and remove
    them.
    '''
    foundOne = False
    for clause in constraints.keys():
        literals = constraints[clause]
        counterLiterals = [-x for x in literals]
        for i in literals:
            for j in counterLiterals:
                if i == j:
                    del constraints[clause]
                    pointers[i].remove(clause)
                    pointers[-i].remove(clause)
                    foundOne = True
                    break
    if foundOne:
        print("Found tautologies and removed corresponding clauses")
    else:
        print("No tautologies found")
                  
    return constraints, pointers  

### Extract features to use as regressor datapoints

In [146]:
def extract_features(literal, constraints, pointers, assignments, logging=False):
    
    ## Look one step ahead
    constraintsNew, pointersNew, assignmentsNew = assign([literal], constraints, pointers, assignments)
    
    ## Ratio of the polarity of this literal before assignment
    positive = len(pointers[literal])
    negative = len(pointers[-literal])
    total = positive+negative
    if total != 0:
        ratio = positive/(positive+negative)
    else:
        ratio = 0
    
    # Clauses that were resolved with this assignment
    clausesResolved = len(constraints) - len(constraintsNew)
    
    # Clauses that became empty with this assignment
    emptyClauses = 0
    for clause in constraintsNew.keys():
        if not constraintsNew[clause]:
            emptyClauses += 1
    
    ## Average clause length with given literal before assignment
    clauses = pointers[literal]
    lengths = [len(constraints[c]) for c in clauses]
    if len(lengths) != 0:
        mean = np.mean(lengths) 
    else:
        mean = 0
    
    if logging:
        features[literal] = {}
        features[literal]["posneg_ratio"] = ratio
        features[literal]["c_res"] = clausesResolved
        features[literal]["empty_c"] = emptyClauses  
        features[literal]["c_len"] = mean
        features[literal]["SAT"] = 0
    else:
        return [ratio, clausesResolved, emptyClauses, mean]
    

### Train linear regressor

In [195]:
def train(file='data.json'):
    X = []
    y = []
    with open(file) as f:  
        data = json.load(f)
        for d in data:
            row = list(d.values())
            X.append(row[:-1])
            y.append(row[-1])
    global regressor
    regressor = LinearRegression().fit(np.array(X), np.array(y))

## DP cases

In [135]:
def empty_clause(constraints):
    for clause in constraints.keys():
        if not constraints[clause]:
            return True
        
    return False

def unit_clauses(constraints): 
    unitLiterals = []
    for clause in constraints.keys():
        if len(constraints[clause])==1:
            unitLiterals.append(next(iter(constraints[clause])))
    return set(unitLiterals)

def unit_propagate(constraints, pointers, assignments):
    '''
    Automatically assign the literal of all unit clauses
    until there are no unit clauses left.
    '''
    unitLiterals = unit_clauses(constraints)
    constraints, pointers, assignments = assign(unitLiterals, constraints, pointers, assignments)
        
    return constraints, pointers, assignments

def pure_literals(pointers):
    pureLiterals = []
    for literal in list(pointers.keys()):
        if -literal not in list(pointers.keys()):
            pureLiterals.append(literal)
        
    return pureLiterals

def random_literal(pointers): 
    literal = random.choice(list(pointers.keys())) 
    return [literal]

### Branch split heuristics

In [175]:
def linreg(constraints, pointers, assignments, b=5): 
    '''
    Linreg performs a prediction on a batch of literals and picks the highest scoring literal.
    '''
    batch = random.sample(list(pointers.keys()), k=b)
    X = np.array([extract_features(literal, constraints, pointers, assignments) for literal in batch])
    scores = regressor.predict(X) # globally available regressor
    literal = batch[np.argmax(scores)]
    return [literal]
    
#Mom's heuristic    
def mom(constraints,pointers,k):
    smallestClasses=[]
    minClass=np.inf
    for clause in constraints.keys():
        if len(constraints[clause])<minClass:
            minClass=len(constraints[clause])
            smallestClasses=[]
            smallestClasses.append(clause)
        elif len(constraints[clause])==minClass:
            smallestClasses.append(clause)
            
    value=0
    maxLiteral=None
    haveSeen=[]
    
    for literal in pointers.keys():
        if literal not in haveSeen and -literal not in haveSeen:
            X=len([x for x in pointers[literal] if x in smallestClasses])
            X_=len([x for x in pointers[-literal] if x in smallestClasses])
            momsValue=(X+X_)*2^k+(X+X_)
            haveSeen.append(literal)
            if momsValue>value:
                value=momsValue
                maxLiteral=literal
    if maxLiteral:
        return [maxLiteral]
    else:
        return []
    
#jeroslow wang heuristic
def jeroSloWang(constraints,pointers):
    j={}
    maxValue=0
    maxLiteral=None
    if len(pointers.keys())==0:
        print('THIS IS NOT GOOD')
    for literal in pointers.keys():
        for clause in pointers[literal]:
            if literal not in j.keys():
                j[literal]=0
            j[literal]+=(2**(-len(constraints[clause])))
        try:
            if j[literal]>maxValue:
                maxValue=j[literal]
                maxLiteral=literal
        except:
            continue
    return [maxLiteral]

## DP main function

In [206]:
def DP(constraints, pointers, assignments, split="random"): 
    '''
    DP main recursive function. Always returns satisfiable
    assignment for all constraints if there is one.
    '''
    
    constraints, pointers, assignments = unit_propagate(constraints, pointers, assignments)
    
    pureLiterals = pure_literals(pointers)
    constraints, pointers, assignments = assign(pureLiterals, constraints, pointers, assignments)
    
    if len(constraints)==0:
        return assignments
   
    if empty_clause(constraints):
        return False
    
    global assignmentCount
    assignmentCount += 1
    
    if split == "random":
        literal = random_literal(pointers)
    if split == "jerow":
        literal = jeroSloWang(constraints, pointers)
    if split == "moms":
        literal = mom(constraints, pointers, 3)
    if split == "linreg":
        literal = linreg(constraints, pointers, assignments, 3)
    
    constraintsUpdated, pointersUpdated, assignmentsUpdated = assign(literal, constraints, pointers, assignments)
    assignmentsFinal = DP(constraintsUpdated, pointersUpdated, assignmentsUpdated)
    
    if assignmentsFinal:
        return assignmentsFinal
    else:
        constraints, pointers, assignments = assign([-next(iter(literal))], constraints, pointers, assignments)
        return DP(constraints, pointers, assignments)

### Script

In [226]:
# train_sudokus = [x for x in range(0,1000) if x%2 == 0]
test_sudokus = [x for x in range(0,1000) if x%2 == 1]

print("Training linear regressor...")
train('data.json')

run_statistics = {}

sudokus1000 = get_sudokus('1000 sudokus.txt')
heuristics = ["random", "moms", "jerow", "linreg"]
    
## Run DP
runs = 2
for run in range(0, runs):
    run_statistics[run] = {}
    for i in random.sample(test_sudokus, 1):
        run_statistics[run][i] = {}
        for h in heuristics:
            run_statistics[run][i][h] = {}
            ## Get board and rules
            
            print("Reading and analyzing...")
            constraints, pointers = sudokus1000[i]
            run_statistics[run][i][h]["beginAssigments"] = len(pointers)
            constraints, pointers = get_constraints_from_dimacs('sudoku-rules.txt', constraints, pointers)
            constraints, pointers = check_tautologies(constraints, pointers)

            print("Starting DP algorithm with "+ h + " splits...")
            global assignmentCount
            assignmentCount = 0
            startTime = time.time()
            assignments = DP(constraints, pointers, [], split=h)
            endTime = time.time()
            if assignments:
                print("Solution found")
                print("Time to complete:", "%2f seconds"%(endTime-startTime))
                boardPositions = [a for a in assignments if a > 0]
                if len(boardPositions) > 81:
                    print("too many assignments for sudoku solution:", len(boardPositions))
                elif len(boardPositions) == 81:
                    print(np.reshape([str(pos)[2] for pos in sorted(boardPositions)], (9,9)))
                print("Branch split choices made:", assignmentCount)
                run_statistics[run][i][h]["assignmentCount"] = assignmentCount
                run_statistics[run][i][h]["runTime"] = (endTime-startTime)
                if h == "linreg":
                    run_statistics[run][i][h]["batchSize"] = 3
            else:
                print("No solution found")
                
        with open('run_statistics.json', 'w') as f:
            json.dump(run_statistics, f)

Training linear regressor...
Reading and analyzing...
No tautologies found
Starting DP algorithm with random splits...
Solution found
Time to complete: 1.541452 seconds
too many assignments for sudoku solution: 85
Branch split choices made: 44
Reading and analyzing...
No tautologies found
Starting DP algorithm with moms splits...
Solution found
Time to complete: 2.334117 seconds
too many assignments for sudoku solution: 83
Branch split choices made: 18
Reading and analyzing...
No tautologies found
Starting DP algorithm with jerow splits...
Solution found
Time to complete: 2.684794 seconds
[['9' '3' '6' '2' '1' '5' '4' '8' '7']
 ['7' '4' '8' '9' '6' '3' '5' '2' '1']
 ['1' '5' '2' '4' '7' '8' '3' '6' '9']
 ['8' '1' '3' '5' '4' '2' '9' '7' '6']
 ['6' '7' '9' '3' '8' '1' '2' '4' '5']
 ['4' '2' '5' '7' '9' '6' '8' '1' '3']
 ['3' '6' '4' '8' '5' '7' '1' '9' '2']
 ['5' '8' '7' '1' '2' '9' '6' '3' '4']
 ['2' '9' '1' '6' '3' '4' '7' '5' '8']]
Branch split choices made: 34
Reading and analyzing.